# CNN으로 MNIST 분류

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1, 1, 28, 28) # 배치크기 x 채널 x 높이 x 너비

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [8]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [9]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [10]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [11]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼치기
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [12]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(777)
if device == 'cuda:0':
    torch.cuda.manual_seed_all(777)
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [16]:
mnist_train = dsets.MNIST(
    root='MNIST_data',
    train=True,
    transform=transforms.ToTensor(),
    download=True)

mnist_test = dsets.MNIST(
    root='MNIST_data',
    train=False,
    transform=transforms.ToTensor(),
    download=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [17]:
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
    )

In [18]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        # fc 한정 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [19]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
total_batch = len(data_loader)
print(f'총 배치 수: {total_batch}')

총 배치 수: 600


In [21]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print(f'Epoch: {epoch+1}, Cost: {avg_cost}')

Epoch: 1, Cost: 0.22571782767772675
Epoch: 2, Cost: 0.06314315646886826
Epoch: 3, Cost: 0.04632938280701637
Epoch: 4, Cost: 0.037502139806747437
Epoch: 5, Cost: 0.031424056738615036
Epoch: 6, Cost: 0.026181066408753395
Epoch: 7, Cost: 0.02198207564651966
Epoch: 8, Cost: 0.018307354301214218
Epoch: 9, Cost: 0.01657949760556221
Epoch: 10, Cost: 0.013524227775633335
Epoch: 11, Cost: 0.01039966382086277
Epoch: 12, Cost: 0.009857735596597195
Epoch: 13, Cost: 0.008125675842165947
Epoch: 14, Cost: 0.006589369848370552
Epoch: 15, Cost: 0.007770432159304619


In [25]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print(f'정확도: {accuracy.item() * 100:.2f}%')

정확도: 98.48%
